# Hyperparameter Tuning Methods Comparision: Example

Cross validation approach based on the following [repo](https://github.com/roamanalytics/roamresearch/tree/master/BlogPosts/Hyperparameter_tuning_comparison)


In [1]:
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 1
%aimport hpt_cmp

In [3]:
from hpt_cmp import *

In [4]:
import matplotlib.pyplot as plt
import numpy as np

from dotmap import DotMap
from sklearn import datasets
#from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
#from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier


In [5]:
# basic sklearn classification DS
dataset = datasets.load_digits() 

# MLPClassifier

Parameters for the MLPClassifier ([full documentation](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier)):
> `hidden_layer_sizes`: tuple <br>
> `activation`: {'identity', 'logistic', 'tanh', *'relu'*} <br>
> `solver`: {‘lbfgs’, ‘sgd’, *‘adam’*} <br>
> `alpha`: float, *0.0001* --- regularization term <br>
> `batch_size`: int, *auto* <br>
> `learning_rate`: {*‘constant’*, ‘invscaling’, ‘adaptive’} <br>
> `learning_rate_init`: double, *0.001* <br>
> `power_t`: double, *0.5* <br>
> `max_iter`: int, *200* <br>
> `shuffle`: bool, *True* <br>
> `random_state` <br>
> `tol`: float, *1e-4* <br>
> `verbose`: bool <br>
> `warm_start`: bool <br>
> `momentum`: float, *0.9* <br>
> `nesterovs_momentum`: bool, *True* <br>
> `early_stopping`: bool, *False* <br>
> `validatino_fraction`: float, *0.1* <br>
> `beta_1`: float, *0.9* <br>
> `beta_2`: float, *0.999* <br>
> `epsilon`: float, *1e-8* <br>
> `n_iter_no_change`: int *10*

In [6]:
param_grid = {
    #'activation': ('relu', 'tanh', 'logistic'),
    #'solver': ('sgd', 'adam'),
    #'hidden_layer_sizes'
    'learning_rate': ('constant', 'invscaling', 'adaptive'),
    'learning_rate_init': [0.01, 0.3, 0.5]
}

In [7]:
# define hpt_obj
hpt_objs = [{
        'name': 'GridSearch',
        'cv': grid_search,
        'param_grid': param_grid,
        'args' : {}
    }]

hpt_objs = [DotMap(obj) for obj in hpt_objs ]

In [8]:
results = cmp_hpt_methods(
    hpt_objs,
    MLPClassifier,
    (dataset.data, dataset.target),
    'log_loss',
    accuracy_score)
results

Start
Here
HTP using GridSearch
<generator object _BaseKFold.split at 0x7f1084bf6888>


/home/tonia/uni/ml/Hyper-Parameter-Tuning/env/lib64/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/tonia/uni/ml/Hyper-Parameter-Tuning/env/lib64/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/tonia/uni/ml/Hyper-Parameter-Tuning/env/lib64/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarn

[{'Model': 'MLPClassifier',
  'Hyper optimization method': 'GridSearch',
  'Test accuracy': [0.9560439560439561,
   0.925414364640884,
   0.958217270194986,
   0.9523809523809523,
   0.9042253521126761],
  'Best Parameters': [{'learning_rate': 'adaptive',
    'learning_rate_init': 0.01},
   {'learning_rate': 'constant', 'learning_rate_init': 0.01},
   {'learning_rate': 'adaptive', 'learning_rate_init': 0.01},
   {'learning_rate': 'invscaling', 'learning_rate_init': 0.01},
   {'learning_rate': 'constant', 'learning_rate_init': 0.01}],
  'Parameters sampled': [9, 9, 9, 9, 9],
  'Cross validation time (in s)': [18.050228357315063,
   15.089909076690674,
   16.60031032562256,
   16.267699718475342,
   15.491988182067871],
  'Mean Test accuracy': 0.9392563790746908,
  'Mean Cross validation time (in s)': 16.3000271320343,
  'Mean Parameters sampled': 9.0}]